## Ufit wrapper for HyperAid

Please first git clone and install Ufit correctly. Then put this Jupyter notebook under its root directory.

Below we start with importing packages

In [1]:
import numpy as np
import higra as hg
from functools import partial
from scipy.cluster.hierarchy import fcluster

from ultrametric.optimization import UltrametricFitting
from ultrametric.data import load_datasets, show_datasets
from ultrametric.graph import build_graph, show_graphs
from ultrametric.utils import Experiments
from ultrametric.evaluation import eval_clustering

# The following line requires that a C++14 compiler be installed
# On Windows, you should probably run
#   c:\Program Files (x86)\Microsoft Visual Studio\2017\Enterprise\VC\Auxiliary\Build\vcvars64.bat
# to properly setup all environment variables
from ultrametric.loss import loss_closest, loss_closest_and_cluster_size, make_triplets, loss_closest_and_triplet, loss_dasgupta

## Ufit

We use their default hyperparameters (epochs=500, lr=0.1). You can fine-tune it.

In [2]:
from scipy.cluster.hierarchy import cophenet
import scipy.spatial.distance as ssd

def closest_ultrametric(X, labeled, y_labeled, graph, edge_weights):
    
    optim = UltrametricFitting(500, 0.1, loss_closest)
    ultrametric = optim.fit(graph, edge_weights)
    
    return hg.bpt_canonical(graph, ultrametric)

def run_Ufit(path,D_path):
    D_metric = np.load(D_path)
    D_input = np.load(path)

    graph, edge_weights = hg.adjacency_matrix_2_undirected_graph(D_input)   
    
    hierarchy = closest_ultrametric(None, None, None, graph, edge_weights)
    
    Z = hg.binary_hierarchy_to_scipy_linkage_matrix(*hierarchy)
    D_output = ssd.squareform(cophenet(Z))

    return ((D_metric-D_output)**2).sum().sum()**(1/2)

## Run Ufit
Remember to specify the path to the saved metric.

In [ ]:
# Load D_input
D_path = "Path to D_target." # Recommend using absolute path.
path = "Path to metric to apply." #  D_metric for HyperAid or D_target for Direct

# Apply
D_loss = run_Ufit(path,D_path)
print(D_loss)